In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10)
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score,f1_score
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import roc_auc_score
import six
import sys
sys.modules['sklearn.externals.six'] = six
import math 


2022-08-04 19:16:49.804492: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-04 19:16:49.804561: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error as mae

In [3]:
df = pd.read_csv("/home/lizard/Documents/Indoor localization Dataset/TrainingData.csv")
df.tail()

,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,WAP520,LONGITUDE,LATITUDE,FLOOR,BUILDINGID,SPACEID,RELATIVEPOSITION,USERID,PHONEID,TIMESTAMP
19932,100,100,100,100,100,100,100,100,100,100,...,100,-7485.4686,4.864875e+06,3,1,1,2,18,10,1371710683
19933,100,100,100,100,100,100,100,100,100,100,...,100,-7390.6206,4.864836e+06,1,2,140,2,18,10,1371710402
19934,100,100,100,100,100,100,100,100,100,100,...,100,-7516.8415,4.864889e+06,3,1,13,2,18,10,1371710921
19935,100,100,100,100,100,100,100,100,100,100,...,100,-7537.3219,4.864896e+06,3,1,113,2,18,10,1371711049
19936,100,100,100,100,100,100,100,100,100,100,...,100,-7536.1658,4.864898e+06,3,1,112,2,18,10,1371711025


In [4]:
X = df.drop(['LONGITUDE',	'LATITUDE',	'FLOOR',	'BUILDINGID',	'SPACEID',	'RELATIVEPOSITION',	'USERID',	'PHONEID',	'TIMESTAMP'],axis=1)
y = df['LATITUDE']

In [5]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit(X)
scaled_data_X=scaler.transform(X)

In [6]:
X_new = SelectKBest(f_regression, k=300).fit_transform(scaled_data_X, y)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.33, random_state=42)

In [8]:
X_new.shape

(19937, 300)

In [9]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

Hyper parameter optimization for Random Forest Regressor

In [10]:
parameters = {'n_estimators': (20,40,60,80,100,120),'min_samples_split' : (2,4,8,10),'max_depth':(5,10,15,20)}
                                                                                                                                     
DT_grid_RF  = RandomizedSearchCV( RandomForestRegressor(), param_distributions = parameters, cv = 3, verbose = 10)
result_RF=DT_grid_RF.fit(X_train,y_train)
  
RF= result_RF.best_estimator_

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV 1/3; 1/10] START max_depth=5, min_samples_split=4, n_estimators=120.........
[CV 1/3; 1/10] END max_depth=5, min_samples_split=4, n_estimators=120;, score=0.839 total time=  11.6s
[CV 2/3; 1/10] START max_depth=5, min_samples_split=4, n_estimators=120.........
[CV 2/3; 1/10] END max_depth=5, min_samples_split=4, n_estimators=120;, score=0.824 total time=  11.2s
[CV 3/3; 1/10] START max_depth=5, min_samples_split=4, n_estimators=120.........
[CV 3/3; 1/10] END max_depth=5, min_samples_split=4, n_estimators=120;, score=0.826 total time=  11.7s
[CV 1/3; 2/10] START max_depth=20, min_samples_split=10, n_estimators=40........
[CV 1/3; 2/10] END max_depth=20, min_samples_split=10, n_estimators=40;, score=0.981 total time=   8.7s
[CV 2/3; 2/10] START max_depth=20, min_samples_split=10, n_estimators=40........
[CV 2/3; 2/10] END max_depth=20, min_samples_split=10, n_estimators=40;, score=0.981 total time=   9.2s
[CV 3/3; 2/10] ST

In [11]:
print(RF)

RandomForestRegressor(max_depth=20, n_estimators=40)


In [12]:
from sklearn import linear_model

Hyper parameter optimization for Ridge Regressor

In [13]:
parameters_ridge_reg = {'alpha': (0.5,1,10),'tol' : (.001,.0001,.01)}
                                                                                                                                     
DT_grid_ridge_reg  = RandomizedSearchCV( linear_model.Ridge(), param_distributions = parameters_ridge_reg, cv = 3, verbose = 10)
result_ridge_reg=DT_grid_ridge_reg.fit(X_train,y_train)
  
ridge_res= result_ridge_reg.best_estimator_

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV 1/3; 1/9] START alpha=0.5, tol=0.001........................................


/home/lizard/.local/lib/python3.8/site-packages/sklearn/model_selection/_search.py:306: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


[CV 1/3; 1/9] END .........alpha=0.5, tol=0.001;, score=0.917 total time=   0.2s
[CV 2/3; 1/9] START alpha=0.5, tol=0.001........................................
[CV 2/3; 1/9] END .........alpha=0.5, tol=0.001;, score=0.915 total time=   0.2s
[CV 3/3; 1/9] START alpha=0.5, tol=0.001........................................
[CV 3/3; 1/9] END .........alpha=0.5, tol=0.001;, score=0.912 total time=   0.2s
[CV 1/3; 2/9] START alpha=0.5, tol=0.0001.......................................
[CV 1/3; 2/9] END ........alpha=0.5, tol=0.0001;, score=0.917 total time=   0.4s
[CV 2/3; 2/9] START alpha=0.5, tol=0.0001.......................................
[CV 2/3; 2/9] END ........alpha=0.5, tol=0.0001;, score=0.915 total time=   0.3s
[CV 3/3; 2/9] START alpha=0.5, tol=0.0001.......................................
[CV 3/3; 2/9] END ........alpha=0.5, tol=0.0001;, score=0.912 total time=   0.2s
[CV 1/3; 3/9] START alpha=0.5, tol=0.01.........................................
[CV 1/3; 3/9] END ..........

In [14]:
print(ridge_res)

Ridge(alpha=10)


Hyper parameter optimization for Lasso Regression

In [15]:
parameters_Lasso_reg = {'alpha': (0.5,1,10),'tol' : (.001,.0001,.01)}
                                                                                                                                     
DT_grid_Lasso_reg  = RandomizedSearchCV( linear_model.Lasso(), param_distributions = parameters_Lasso_reg, cv = 3, verbose = 10)
result_lasso_reg=DT_grid_Lasso_reg.fit(X_train,y_train)
  
lasso_res= result_lasso_reg.best_estimator_

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV 1/3; 1/9] START alpha=0.5, tol=0.001........................................


/home/lizard/.local/lib/python3.8/site-packages/sklearn/model_selection/_search.py:306: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


[CV 1/3; 1/9] END .........alpha=0.5, tol=0.001;, score=0.913 total time=   0.5s
[CV 2/3; 1/9] START alpha=0.5, tol=0.001........................................
[CV 2/3; 1/9] END .........alpha=0.5, tol=0.001;, score=0.912 total time=   0.6s
[CV 3/3; 1/9] START alpha=0.5, tol=0.001........................................
[CV 3/3; 1/9] END .........alpha=0.5, tol=0.001;, score=0.910 total time=   0.6s
[CV 1/3; 2/9] START alpha=0.5, tol=0.0001.......................................
[CV 1/3; 2/9] END ........alpha=0.5, tol=0.0001;, score=0.913 total time=   0.8s
[CV 2/3; 2/9] START alpha=0.5, tol=0.0001.......................................
[CV 2/3; 2/9] END ........alpha=0.5, tol=0.0001;, score=0.912 total time=   0.8s
[CV 3/3; 2/9] START alpha=0.5, tol=0.0001.......................................
[CV 3/3; 2/9] END ........alpha=0.5, tol=0.0001;, score=0.910 total time=   0.9s
[CV 1/3; 3/9] START alpha=0.5, tol=0.01.........................................
[CV 1/3; 3/9] END ..........

In [16]:
print(lasso_res)

Lasso(alpha=0.5, tol=0.001)


In [17]:
from sklearn.neighbors import KNeighborsRegressor

Hyperparameter optimization of KNN Regressor

In [43]:
parameters_KNN_reg = {'n_neighbors': (5,10,25,50),'leaf_size' : (30,60,90,105)}
                                                                                                                                     
DT_grid_KNN_reg  = RandomizedSearchCV( KNeighborsRegressor(), param_distributions = parameters_KNN_reg, cv = 3, verbose = 10)
result_KNN_reg=DT_grid_KNN_reg.fit(X_train,y_train)
  
KNN_res= result_KNN_reg.best_estimator_

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV 1/3; 1/10] START leaf_size=90, n_neighbors=25...............................
[CV 1/3; 1/10] END leaf_size=90, n_neighbors=25;, score=0.974 total time=   0.6s
[CV 2/3; 1/10] START leaf_size=90, n_neighbors=25...............................
[CV 2/3; 1/10] END leaf_size=90, n_neighbors=25;, score=0.974 total time=   0.4s
[CV 3/3; 1/10] START leaf_size=90, n_neighbors=25...............................
[CV 3/3; 1/10] END leaf_size=90, n_neighbors=25;, score=0.973 total time=   0.4s
[CV 1/3; 2/10] START leaf_size=60, n_neighbors=50...............................
[CV 1/3; 2/10] END leaf_size=60, n_neighbors=50;, score=0.964 total time=   0.5s
[CV 2/3; 2/10] START leaf_size=60, n_neighbors=50...............................
[CV 2/3; 2/10] END leaf_size=60, n_neighbors=50;, score=0.962 total time=   0.5s
[CV 3/3; 2/10] START leaf_size=60, n_neighbors=50...............................
[CV 3/3; 2/10] END leaf_size=60, n_neighbors=50;

In [44]:
print(KNN_res)

KNeighborsRegressor(leaf_size=105)


In [21]:
from sklearn.svm import SVR

Hyper parameter optimization of svr

In [22]:
parameters_SVR_reg = {'epsilon': (0.5,1,10),'C' : (1,5,10,50)}
                                                                                                                                     
DT_grid_svr_reg  = RandomizedSearchCV(  SVR(), param_distributions = parameters_SVR_reg, cv = 3, verbose = 10)
result_svr_reg=DT_grid_svr_reg.fit(X_train,y_train)
  
svr_res= result_svr_reg.best_estimator_

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV 1/3; 1/10] START C=50, epsilon=10...........................................
[CV 1/3; 1/10] END ............C=50, epsilon=10;, score=0.962 total time=  14.6s
[CV 2/3; 1/10] START C=50, epsilon=10...........................................
[CV 2/3; 1/10] END ............C=50, epsilon=10;, score=0.962 total time=  13.7s
[CV 3/3; 1/10] START C=50, epsilon=10...........................................
[CV 3/3; 1/10] END ............C=50, epsilon=10;, score=0.960 total time=  12.1s
[CV 1/3; 2/10] START C=1, epsilon=10............................................
[CV 1/3; 2/10] END .............C=1, epsilon=10;, score=0.792 total time=  22.9s
[CV 2/3; 2/10] START C=1, epsilon=10............................................
[CV 2/3; 2/10] END .............C=1, epsilon=10;, score=0.799 total time=  25.2s
[CV 3/3; 2/10] START C=1, epsilon=10............................................
[CV 3/3; 2/10] END .............C=1, epsilon=10;

In [23]:
print(svr_res)

SVR(C=50, epsilon=0.5)


Random Forest Main Train and Result

In [25]:
reg_RF = RandomForestRegressor(max_depth=20, n_estimators=40, random_state=0)
reg_RF.fit(X_train, y_train)

RandomForestRegressor(max_depth=20, n_estimators=40, random_state=0)

In [26]:
reg_RF_pred= reg_RF.predict(X_test)
rmse_reg_RF = math.sqrt(mean_squared_error(y_test, reg_RF_pred, squared=False))
print(rmse_reg_RF)

3.0209633570577816


In [27]:
mae_reg_RF= mae(y_test, reg_RF_pred)
print(mae_reg_RF)

4.9546149450570836


Training and testing with Ridge Regressor

In [28]:
reg_ridge = linear_model.Ridge(alpha=10, random_state=0)
reg_ridge.fit(X_train, y_train)

Ridge(alpha=10, random_state=0)

In [29]:
reg_ridge_pred= reg_ridge.predict(X_test)
rmse_reg_ridge = math.sqrt(mean_squared_error(y_test, reg_ridge_pred, squared=False))
print(rmse_reg_ridge)

4.448313226879736


In [30]:
mae_reg_ridge= mae(y_test, reg_ridge_pred)
print(mae_reg_ridge)

14.944311918484438


Training and testing with Lasso Regressor

In [40]:
reg_Lasso = linear_model.Lasso(alpha=0.5, tol=0.001, random_state=0)
reg_Lasso.fit(X_train, y_train)

Lasso(alpha=0.5, random_state=0, tol=0.001)

In [41]:
reg_lasso_pred= reg_Lasso.predict(X_test)
rmse_reg_lasso = math.sqrt(mean_squared_error(y_test, reg_lasso_pred, squared=False))
print(rmse_reg_lasso)

4.49290083820079


In [42]:
mae_reg_lasso= mae(y_test, reg_lasso_pred)
print(mae_reg_lasso)

15.345904668798177


Training and Testing with K Neigbour Regressor

In [45]:
reg_KNN = KNeighborsRegressor(leaf_size=105)
reg_KNN.fit(X_train, y_train)

KNeighborsRegressor(leaf_size=105)

In [46]:
reg_KNN_pred= reg_KNN.predict(X_test)
rmse_reg_KNN = math.sqrt(mean_squared_error(y_test, reg_KNN_pred, squared=False))
print(rmse_reg_KNN)

2.922805347470722


In [47]:
mae_reg_KNN= mae(y_test, reg_KNN_pred)
print(mae_reg_KNN)

4.247143005650762


Training and Testing with SVR

In [48]:
reg_SVR = SVR(C=50, epsilon=0.5)
reg_SVR.fit(X_train, y_train)

SVR(C=50, epsilon=0.5)

In [49]:
reg_SVR_pred= reg_SVR.predict(X_test)
rmse_reg_SVR = math.sqrt(mean_squared_error(y_test, reg_SVR_pred, squared=False))
print(rmse_reg_SVR)

3.2975904058460297


In [50]:
mae_reg_SVR= mae(y_test, reg_SVR_pred)
print(mae_reg_SVR)

5.629756036485577
